# Dictionaries and Sets

Dictionaries are central to Python's implementation.

Under the hood dictionaries and sets are hash tables.

In [ ]:
# builtin functions live in a dict:
__builtins__.__dict__

In Python, an object is hashable if it has a hash value that never changes during its lifetime. It must a \__hash__() method, and a \__eq__() method. Hashable objects which compare equal must have equal hashes.

Atomic immutable types, such as str, bytes, numeric types, are all hashable. A tuple is hashable if all its elements are hashable.

In [ ]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('three', 3), ('one', 1)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

dict.setdefault(key, default) will return the value assocaited with the key, or the default value if key is not found

## collections.defaultdict

collections.defauldict can be given a callable that produces a default value when \__getitem__ is passed a nonexistent key.

This callable is held in an instance attribute called default_factory

mappings in python deal with missing keys using the \__missing__ method

\__getitem__ will call \__missing__ if key not found

In Python 3 a search like k in my_dict.keys() is very efficient. Because dict.keys() returns a view, which is like a set, and set containment checks are very fast.

## Variations of dict

###### collections.OrderedDict

Maintains the insertion order of keys. popitem() returns the last item by default.
popitem(last=False) returns the first item

###### collections.ChainMap

Holds a list of mappings that be searched as one.
The lookup happens on each mapping in order, and succeeds if the key is found in any of them.

In [ ]:
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))
#pylookup

###### collections.Counter

Mapping that holds an integer count for each key.
Updating an existing key adds to its count.

In [ ]:
ct = collections.Counter('abaracadebaraaaa')

###### collections.UserDict

Designed to be subclassed

## Subclassing UserDict

collections.UserDict subclasses from MutableMapping, which subclasses from Mapping.

## Sets

Set elements must be hashable. set type is not hashable, but frozenset is.

infix operators can be used. a | b for union, a & b for intersection, and a - b for difference

Counting occurences of needles in a haystack: len(needles & haystack)

There is no literal notation for empty set. e.g. {} does not mean empty set, it means empty dictionary.

So set() must be used for that.

Building a set with {1, 2, 3} is faster than set([1, 2, 3]), because the latter requires looking up set(), building a list, and passing it to the constructur.
While {1, 2, 3} runs specialiszed BUILT_SET bytecode.

In [ ]:
import dis
dis.dis('{1}')

In [ ]:
dis.dis('set([1])')

frozenset is an immutable set

infix operators require that both operands be sets.
All other set methods can take one or more iterables.

Producing the union of four collections can be done with a.union(b, c, d) - where only a must be a set. The others can be iterables.

Sets also support augmented assignment operators such as |=

Sets in Python support all ideal mathematical operations

https://docs.python.org/2/library/stdtypes.html#set

Or see page 88 of Fluent Ptyhon book

Dicts and sets are very fast in Python

## Hash tables in Dictionaries

There is a bucket for each item. Each bucket contains two fields, a reference to the key, and a reference to the value of the item.

Python tries to keep a third of the bucket empty. Crowded hashtables are copied to a new location with more space.

Putting an item in a hashtable first requires getting the hash value of the item's key, using the hash() function.

Equal items must have equals hash values. 1 == 1.0 is ture, hash(1) == hash(1.0) must also be true

The hash value of an int that fits in a machine word is the value of the int itself

Hash values are scattered around the index space as much as possible. Ideally, objects that are similar but not equal have very very different hashes.

###### How the has table algorithm works in Python

To get the value of dict[key], Python calls hash(key). It uses part of the hash value of key to locate a bucket in the hashtable. 
If the found bucket is empty, a KeyError is raised
Otherwise, a pair found_key:found_value is found. If search_key == found_key, the value is returned.
Otherwise, a hash collision occurs. The algorithm takes different bits of the hash, combines them in some way, and usese that to lookup a different bucket. 
If the newly looked up bucket is empty, a KeyError is raised. Otherwise, either keys match, or there is another hash collision and the collision resolution process repeats.

See figure 3-3 on page 94 for flowchart.

###### What this means for dicts practically

Keys must be hashable. Meaning the key object must:

1. support the hash() function
2. support equality via an \__eq__() method
3. if a == b is True, then hash(a) == hash(b) must also be True

Since dicts use hashtables internally, they must be sparse, and are thus not very space effecient.
For large records, using a list of tuples or namedTuples is more efficient a list of dicts, as each dict requires storing field names, and there is extra overhead for each hash table.

*Optimision is the altar where maintainability is sacrificed.*

If collisions occur, it will change the ordering of the keys. This happens when a second bucket is looked up using different parts of the hash.

Dicts are compared based on pair containment alone, and not on ordering.

When a new item is added to a dict, the Python interpreter may deicde the hash table needs to grow in size. So a bigger hash table is created, and all entries added into it.

During this process, new hash collisions may occur, so it is likely that the keys will be ordered differently.

Modifying dict contents while iterating over it is a bad idea, becuase the order of the dict may chance. In which case you miss scanning certain items.

Better to read the dict completely, collecting the needed additions in a seperate dict, then updating with the first one.

In Python 3 .keys(), .items(), and .values() return dictionary views. These are dynamic and will always refelct any chances to the dict.

## Hash tables and sets

set and frozenset are also implemented with a hashtable, but each buckets holds a single reference to the item.

All of the above notes on dicts apply to sets:

* Set elements must be hashable
* Sets have significant memory overhead
* Memberhsip testing is very effecient
* Element ordering depends on insertion order
* Adding elements may change the order of elements in the set

## Chapter Summary

Most Python mappings have an update method. It is very powerful, allowing for bulk insertion or overwriting from either another mapping, an iterable providing (key, value) pairs, and from keyword arguments. 
Mapping constuctors use update internally, allowing mappings to be initialised from mappings, iterables, and kwargs.

The \__missing__ method in the mappings API lets use customise what happens when a key is not found.

MappingProxyType from the types module lets you create immutable mappings.

hashtable implemention underyling dicts and sets is extremely fast, but pays a price in memory.